# 指値厚いとこのちょい前に指し太郎

In [1]:
import datetime
from math import floor
import pprint
import schedule
from statistics import mean
from statistics import median
from statistics import StatisticsError
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import oandapyV20.endpoints.forexlabs as labs

sys.path.append('../')
from oandapyV21 import *

# OANDA API v20の口座IDとAPIトークン(房林蛇ver)
accountID = "101-009-13234797-001"
access_token = "9739ebcc49088322e07e334ad3697b04-16314025c95d08a3d40882b027316211"

In [2]:
def get_orderbook(date='', inst='USD_JPY'):
    api = API(access_token=access_token, environment="practice")
    if date != '':
        year, month, day, hour, minute, second = map(int, date.split('-'))
        fmt = '%Y-%m-%dT%H:%M:00.000000Z'
        time = datetime.datetime(year=year, month=month, day=day,
                                  hour=hour, minute=minute, second=second).strftime(fmt)
        params = {
            'time': time,
        }
        r = instruments.InstrumentsOrderBook(instrument=inst, params=params)
    else:
        r = instruments.InstrumentsOrderBook(instrument=inst)
    api.request(r)
    return r

In [77]:
r = get_orderbook(inst='USD_JPY')
buckets = r.response['orderBook']['buckets']
price = []
long = []
short = []
for i in range(len(buckets)):
    price.extend([buckets[i]['price']])
    long.extend([float(buckets[i]['longCountPercent'])])
    short.extend([float(buckets[i]['shortCountPercent'])])
order_df = pd.DataFrame(columns=['price', 'long_percent', 'short_percent'],
                        data={'price': price, 'long_percent': long, 'short_percent': short})

In [79]:
order_df.sort_values(by='long_percent', ascending=False).head(10)

,price,long_percent,short_percent
667,109.700,2.3135,0.1151
182,54.900,0.7328,0.0000
668,109.750,0.6480,0.5814
0,0.000,0.5875,0.0121
639,108.300,0.4845,0.1393
523,102.500,0.4784,0.0242
753,114.000,0.4421,0.1696
1341,1300.000,0.4179,0.0000
666,109.650,0.3755,0.1575
623,107.500,0.3634,0.1090


In [80]:
order_df.sort_values(by='short_percent', ascending=False).head(10)

,price,long_percent,short_percent
1030,131.000,0.0000,6.0380
669,109.800,0.1877,5.3234
674,110.050,0.1635,1.6594
673,110.000,0.3391,1.5201
677,110.200,0.1332,1.5141
672,109.950,0.2241,1.4777
678,110.250,0.1332,1.4414
675,110.100,0.1938,0.8115
676,110.150,0.1817,0.7994
721,112.400,0.0303,0.7631


In [ ]:
order_df.tail()

In [76]:
r = get_orderbook(inst='GBP_USD')
r.response

{'orderBook': {'instrument': 'GBP_USD',
  'time': '2020-02-05T14:00:00Z',
  'unixTime': '1580911200',
  'price': '1.30092',
  'bucketWidth': '0.00050',
  'buckets': [{'price': '0.00000',
    'longCountPercent': '0.1447',
    'shortCountPercent': '0.0814'},
   {'price': '0.00050',
    'longCountPercent': '0.0181',
    'shortCountPercent': '0.0090'},
   {'price': '0.00100',
    'longCountPercent': '0.0543',
    'shortCountPercent': '0.0090'},
   {'price': '0.00150',
    'longCountPercent': '0.0090',
    'shortCountPercent': '0.0090'},
   {'price': '0.00200',
    'longCountPercent': '0.0090',
    'shortCountPercent': '0.0000'},
   {'price': '0.00250',
    'longCountPercent': '0.0271',
    'shortCountPercent': '0.0181'},
   {'price': '0.00300',
    'longCountPercent': '0.0271',
    'shortCountPercent': '0.0090'},
   {'price': '0.00350',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0090'},
   {'price': '0.00400',
    'longCountPercent': '0.0090',
    'shortCountPercent': '0

In [5]:
# ==============================================================================
# brief        オープンオーダー、オープンポジションの描写
#
# author       たっきん
#
# 事前準備 :
#     oandapyV20のインストール (pip install oandapyV20)
#     Bokehのインストール（conda install bokeh）
# ==============================================================================

import copy
import datetime
from bokeh.plotting import figure, show
from oandapyV20 import API
from bokeh.layouts import gridplot
from datetime import timedelta

from bokehlib import bokeh_common as bc
from fxx import oanda_common as oc
from fxx import your_account as ya
import oandapyV20.endpoints.instruments as instruments
import pandas as pd


class OrderBook(object):
    """ OrderBook - オーダーブック定義クラス。"""

    def __init__(self, granularity):
        """"コンストラクタ"""

        self.__BUCKETS = "buckets"

        self.__ORD_BOOK = "orderBook"
        self.__PSI_BOOK = "positionBook"
        self.__PRICE = "price"
        self.__LONG = "longCountPercent"
        self.__SHORT = "shortCountPercent"

        self.__TIME = "time"
        self.__CUR_PRICE = "price"
        self.__BUCKET_WIDTH = "bucketWidth"

        self.__WIDE = 12 * 60 * 60 * 1000  # half day in ms
        self.__WIDE_SCALE = 0.2

        self.__DT_FMT = "%Y-%m-%dT%H:%M:00Z"
        self.__GRANULARITY = granularity

        self.__CUT_TH = 50  # 現レートから上下何本残すか
        self.__X_AXIS_MAX = 2.5  # X軸レンジ

        self.__BG_COLOR = "#2e2e2e"
        self.__BAR_R_COLOR = "#00A4BD"
        self.__BAR_L_COLOR = "#FF8400"
        self.__CURPRI_COLOR = "#7DA900"

        self.__ord_df = []
        self.__ord_curpri = 0

        self.__psi_df = []
        self.__psi_curpri = 0

        self.__api = api

    def getInstrumentsOrderBook(self, instrument, dt):

        params = {
            "time": dt.strftime(self.__DT_FMT),
        }

        # APIへ過去データをリクエスト
        ic = instruments.InstrumentsOrderBook(instrument=instrument,
                                              params=params)
        self.__api.request(ic)

        self.__data = []
        for raw in ic.response[self.__ORD_BOOK][self.__BUCKETS]:
            self.__data.append([float(raw[self.__PRICE]),
                                float(raw[self.__LONG]),
                                float(raw[self.__SHORT])])

        # リストからデータフレームへ変換
        df = pd.DataFrame(self.__data)
        df.columns = [self.__PRICE,
                      self.__LONG,
                      self.__SHORT]
        df = df.set_index(self.__PRICE).sort_index(ascending=False)
        # date型を整形する
        time = pd.to_datetime(self.__changeDateTimeFmt(
            ic.response[self.__ORD_BOOK][self.__TIME]))
        cur_price = float(ic.response[self.__ORD_BOOK][self.__CUR_PRICE])
        bucket_width = float(ic.response[self.__ORD_BOOK][self.__BUCKET_WIDTH])

        print(df)
        print(bucket_width)

        print(time)
        print(cur_price)
        idx_th = bucket_width * self.__CUT_TH
        self.__ord_df = df[(df.index > cur_price - idx_th)
                           & (df.index < cur_price + idx_th)]
        self.__ord_curpri = cur_price

    def getInstrumentsPositionBook(self, instrument, dt):

        params = {
            "time": dt.strftime(self.__DT_FMT),
        }

        # APIへ過去データをリクエスト
        ic = instruments.InstrumentsPositionBook(instrument=instrument,
                                                 params=params)
        self.__api.request(ic)

        self.__data = []
        for raw in ic.response[self.__PSI_BOOK][self.__BUCKETS]:
            self.__data.append([float(raw[self.__PRICE]),
                                float(raw[self.__LONG]),
                                float(raw[self.__SHORT])])

        # リストからデータフレームへ変換
        df = pd.DataFrame(self.__data)
        df.columns = [self.__PRICE,
                      self.__LONG,
                      self.__SHORT]
        df = df.set_index(self.__PRICE).sort_index(ascending=False)
        # date型を整形する
        time = pd.to_datetime(self.__changeDateTimeFmt(
            ic.response[self.__PSI_BOOK][self.__TIME]))
        cur_price = float(ic.response[self.__PSI_BOOK][self.__CUR_PRICE])
        bucket_width = float(ic.response[self.__PSI_BOOK][self.__BUCKET_WIDTH])

        print(df)
        print(bucket_width)

        print(time)
        print(cur_price)
        idx_th = bucket_width * self.__CUT_TH
        self.__psi_df = df[(df.index > cur_price - idx_th)
                           & (df.index < cur_price + idx_th)]
        self.__psi_curpri = cur_price

    def drawPositionOrderBook(self, fig_width=500):

        set_tools = bc.ToolType.gen_str(bc.ToolType.XPAN,
                                        bc.ToolType.WHEEL_ZOOM,
                                        bc.ToolType.BOX_ZOOM,
                                        bc.ToolType.RESET,
                                        bc.ToolType.SAVE)

        df = copy.copy(self.__ord_df)
        # --------------- メインfigure ---------------
        plt1 = figure(
            plot_height=fig_width,
            plot_width=fig_width,
            x_range=(-self.__X_AXIS_MAX, self.__X_AXIS_MAX),
            tools=set_tools,
            title="Order Book example",
            background_fill_color=self.__BG_COLOR
        )
        plt1.grid.grid_line_alpha = 0.3

        df_up = df[self.__LONG][(df.index > self.__ord_curpri)]
        df_lo = -df[self.__SHORT][(df.index < self.__ord_curpri)]
        df_right = pd.concat([df_up, df_lo])

        df_up = -df[self.__SHORT][(df.index > self.__ord_curpri)]
        df_lo = df[self.__LONG][(df.index < self.__ord_curpri)]
        df_left = pd.concat([df_up, df_lo])

        plt1.hbar(y=df.index, height=0.03, left=df_right,
                  right=0, color=self.__BAR_R_COLOR)
        plt1.hbar(y=df.index, height=0.03, left=df_left,
                  right=0, color=self.__BAR_L_COLOR)
        plt1.line(x=[-self.__X_AXIS_MAX, self.__X_AXIS_MAX],
                  y=[self.__ord_curpri, self.__ord_curpri],
                  color=self.__CURPRI_COLOR, line_width=3)

        plt1.xaxis.axis_label = "Count Percent[%]"
        plt1.yaxis.axis_label = "Price"

        df = copy.copy(self.__psi_df)
        # --------------- メインfigure ---------------
        plt2 = figure(
            plot_height=fig_width,
            plot_width=fig_width,
            x_range=(-self.__X_AXIS_MAX, self.__X_AXIS_MAX),
            tools=set_tools,
            title="Position Book example",
            background_fill_color=self.__BG_COLOR
        )
        plt2.grid.grid_line_alpha = 0.3

        df_up = df[self.__LONG][(df.index > self.__psi_curpri)]
        df_lo = -df[self.__SHORT][(df.index < self.__psi_curpri)]
        df_right = pd.concat([df_up, df_lo])

        df_up = -df[self.__SHORT][(df.index > self.__psi_curpri)]
        df_lo = df[self.__LONG][(df.index < self.__psi_curpri)]
        df_left = pd.concat([df_up, df_lo])

        plt2.hbar(y=df.index, height=0.03, left=df_right,
                  right=0, color=self.__BAR_R_COLOR)
        plt2.hbar(y=df.index, height=0.03, left=df_left,
                  right=0, color=self.__BAR_L_COLOR)
        plt2.line(x=[-self.__X_AXIS_MAX, self.__X_AXIS_MAX],
                  y=[self.__psi_curpri, self.__psi_curpri],
                  color=self.__CURPRI_COLOR, line_width=3)

        plt2.xaxis.axis_label = "Count Percent[%]"
        plt2.yaxis.axis_label = "Price"

        # make a grid
        grid = gridplot([[plt1, plt2]])

        show(grid)

    def __changeDateTimeFmt(self, dt):
        """"日付フォーマットの変換メソッド
        引数:
            dt (str): DT_FMT形式でフォーマットされた日付
        戻り値:
            tf_dt (str): 変換後の日付
        """
        tdt = datetime.datetime.strptime(dt, self.__DT_FMT)

        return tdt

In [6]:
if True:
    cs = OrderBook(oc.OandaGrn.D)

    instrument = oc.OandaIns.USD_JPY

    dt = datetime.datetime(year=2019, month=5, day=18,
                           hour=21, minute=40, second=0)
    dttk = dt - timedelta(hours=9)

    cs.getInstrumentsOrderBook(instrument, dttk)
    cs.getInstrumentsPositionBook(instrument, dttk)
    cs.drawPositionOrderBook(500)

              longCountPercent  shortCountPercent
price                                            
9.223372e+15            0.0000             0.0229
1.000000e+08            0.0000             0.0076
2.489500e+06            0.0076             0.0000
1.000022e+06            0.0000             0.0076
1.000000e+06            0.0076             0.0153
1.261690e+05            0.0076             0.0000
1.230560e+05            0.0000             0.0076
1.133000e+05            0.0000             0.0076
1.126600e+05            0.0076             0.0000
1.113690e+05            0.0000             0.0076
1.102710e+05            0.0000             0.0076
1.098170e+05            0.0000             0.0076
1.095220e+05            0.0000             0.0076
1.001110e+05            0.0000             0.0076
1.000000e+05            0.0000             0.0153
7.020000e+04            0.0000             0.0076
5.000000e+04            0.0000             0.0153
1.999995e+04            0.0000             0.0076
